In [1]:
# ==== HARD BLOCK RAPIDS (fixes cudaErrorInsufficientDriver) ====
import os, sys, textwrap, pathlib, shutil

# 1) Ensure no old stubs
for pkg in ("cuml","cudf","cupy","rmm"):
    shutil.rmtree(pkg, ignore_errors=True)

# 2) Create stub packages that raise ImportError on import
for pkg in ("cuml","cudf","cupy","rmm"):
    pathlib.Path(pkg).mkdir(exist_ok=True)
    with open(f"{pkg}/__init__.py","w") as f:
        f.write('raise ImportError("Stub: {pkg} not available")\n')

# 3) Put current dir first on sys.path so stubs shadow any preinstalled RAPIDS
if "" not in sys.path:
    sys.path.insert(0, "")

print("RAPIDS stubs installed & shadowing real packages.")



# 2) Install PyCaret (CPU-safe) + deps
!pip -q install -U pip setuptools wheel
!pip -q install -U pycaret pandas scikit-learn


# --- Imports & environment info ---
import sys, platform, numpy as np, pandas as pd
from datetime import datetime
print("Python:", sys.version.split()[0])
import pycaret
print("PyCaret:", pycaret.__version__)
print("Run time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
# Load Dry Bean dataset from Kaggle
import pandas as pd
path = "../input/dry-bean-dataset/Dry_Bean_Dataset/Dry_Bean_Dataset.xlsx"
df = pd.read_excel(path)
df.head()
# --- PyCaret multiclass setup ---

from pycaret.classification import *

clf = setup(
    data=df,
    target="Class",
    session_id=42,
    normalize=True,
    feature_selection=True,
    remove_multicollinearity=True,
    fix_imbalance=True,
    use_gpu=False,   # stay on CPU
    fold=5
)

# --- Compare models ---
best_models = compare_models(n_select=5, sort="Accuracy")

# --- Tune top model ---
tuned_best = tune_model(best_models[0])

# --- Blend and finalize ---
blended = blend_models(best_models)
final_model = finalize_model(blended)

# --- Evaluate model performance ---
evaluate_model(final_model)


RAPIDS stubs installed & shadowing real packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.2 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.

,Description,Value
0,Session id,42
1,Target,Class
2,Target type,Multiclass
3,Target mapping,"BARBUNYA: 0, BOMBAY: 1, CALI: 2, DERMASON: 3, HOROZ: 4, SEKER: 5, SIRA: 6"
4,Original data shape,"(13611, 17)"
5,Transformed data shape,"(21458, 4)"
6,Transformed train set shape,"(17374, 4)"
7,Transformed test set shape,"(4084, 4)"
8,Numeric features,16
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9049,0.9881,0.9049,0.9054,0.9049,0.8851,0.8852,2.1900
catboost,CatBoost Classifier,0.9034,0.9904,0.9034,0.9043,0.9036,0.8833,0.8834,12.5300
gbc,Gradient Boosting Classifier,0.9029,0.0000,0.9029,0.9035,0.9029,0.8827,0.8828,10.1740
rf,Random Forest Classifier,0.9028,0.9870,0.9028,0.9033,0.9028,0.8826,0.8826,2.9660
lightgbm,Light Gradient Boosting Machine,0.8998,0.9894,0.8998,0.9005,0.8999,0.8789,0.8790,149.6920
lr,Logistic Regression,0.8997,0.0000,0.8997,0.9006,0.8998,0.8788,0.8790,3.4180
qda,Quadratic Discriminant Analysis,0.8972,0.0000,0.8972,0.8991,0.8974,0.8760,0.8764,1.8220
xgboost,Extreme Gradient Boosting,0.8968,0.9890,0.8968,0.8976,0.8969,0.8753,0.8754,2.5540
knn,K Neighbors Classifier,0.8965,0.9722,0.8965,0.8974,0.8967,0.8750,0.8751,1.8860
lda,Linear Discriminant Analysis,0.8833,0.0000,0.8833,0.8896,0.8836,0.8593,0.8607,1.7520


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8998,0.9892,0.8998,0.9010,0.8996,0.8788,0.8791
1,0.8788,0.9822,0.8788,0.8798,0.8787,0.8538,0.8541
2,0.8877,0.9838,0.8877,0.8880,0.8873,0.8644,0.8646
3,0.9029,0.9891,0.9029,0.9056,0.9029,0.8827,0.8833
4,0.8976,0.9892,0.8976,0.8991,0.8973,0.8764,0.8767
Mean,0.8934,0.9867,0.8934,0.8947,0.8932,0.8712,0.8715
Std,0.0089,0.0031,0.0089,0.0094,0.0089,0.0106,0.0107


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9134,0.9921,0.9134,0.9139,0.9133,0.8953,0.8954
1,0.8867,0.9876,0.8867,0.8873,0.8868,0.8632,0.8633
2,0.8940,0.9864,0.8940,0.8948,0.8940,0.8720,0.8722
3,0.9176,0.9915,0.9176,0.9190,0.9180,0.9003,0.9005
4,0.9192,0.9930,0.9192,0.9191,0.9191,0.9023,0.9023
Mean,0.9062,0.9901,0.9062,0.9068,0.9062,0.8866,0.8867
Std,0.0133,0.0026,0.0133,0.0132,0.0133,0.0159,0.0159


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…